In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import argparse
import os

import numpy as np
import torch
from torch.nn.modules.loss import CrossEntropyLoss
from torch.utils.data.dataloader import DataLoader
from tqdm import tqdm
import random

from handwriting_recognition.label_converter import LabelConverter
from handwriting_recognition.model.model import HandwritingRecognitionModel
from handwriting_recognition.modelling_utils import get_image_model
from handwriting_recognition.utils import TrainingConfig, get_dataset_folder_path
from handwriting_recognition.dataset import HandWritingDataset
from pathlib import Path
from handwriting_recognition.modelling_utils import get_device
from handwriting_recognition.eval import cer, wer

torch.backends.cudnn.benchmark = True

/home/faraz/.cache/pypoetry/virtualenvs/handwriting-recognition-05HTZ8rb-py3.11/lib/python3.11/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [3]:
CONFIG_NAME = "resnet_config"
MODEL_TO_TEST = "best"
MODEL_FILE = Path(get_dataset_folder_path()).parent.joinpath("model_outputs", CONFIG_NAME, MODEL_TO_TEST)

In [4]:
config_path = Path(get_dataset_folder_path()).parent.joinpath("handwriting_recognition", "configs", CONFIG_NAME).with_suffix(".json")
config = TrainingConfig.from_path(config_path=config_path)

In [5]:
data_train = HandWritingDataset(
    data_path=get_dataset_folder_path() / "pre_processed" / "train.csv",
    img_size=config.feature_extractor_config.input_size,
)

data_val = HandWritingDataset(
    data_path=get_dataset_folder_path() / "pre_processed" / "validation.csv",
    img_size=config.feature_extractor_config.input_size,
)

data_test = HandWritingDataset(
    data_path=get_dataset_folder_path() / "pre_processed" / "test.csv",
    img_size=config.feature_extractor_config.input_size,
)

In [6]:
saved_model = torch.load(MODEL_FILE)

In [7]:
if "scheduler_config" not in saved_model['config']:
    saved_model['config']['scheduler_config'] = None
if "beta1" not in saved_model['config']['optim_config']:
    saved_model['config']['optim_config']['beta1'] = 0.95
if "beta2" not in saved_model['config']['optim_config']:
    saved_model['config']['optim_config']['beta2'] = 0.99

In [8]:
config = TrainingConfig(**saved_model['config'])

In [9]:
image_model = get_image_model(model_name=config.feature_extractor_config.model_name)
model = HandwritingRecognitionModel(image_feature_extractor=image_model, training_config=config)

In [10]:
model.load_state_dict(saved_model["state"])

<All keys matched successfully>

In [11]:
converter = LabelConverter(character_set=saved_model['character_set'], max_text_length=saved_model['max_text_length'])

In [12]:
model = model.to(get_device())
model = model.eval()

In [13]:
BATCH_SIZE = 64

In [14]:
train_loader = DataLoader(
    data_train,
    batch_size=BATCH_SIZE,
    shuffle=False,
    pin_memory=False,
    drop_last=False,
)

val_loader = DataLoader(
    data_val,
    batch_size=BATCH_SIZE,
    shuffle=False,
    pin_memory=False,
    drop_last=False,
)

test_loader = DataLoader(
    data_test,
    batch_size=BATCH_SIZE,
    shuffle=False,
    pin_memory=False,
    drop_last=False,
)

In [15]:
images, labels = next(iter(train_loader))

In [16]:
images.shape

torch.Size([64, 1, 224, 224])

In [17]:
text, length = converter.encode(labels)
text = text.to(device=get_device())
images = images.to(device=get_device())

In [18]:
preds = model(x=images, y=text[:, :-1], is_train=False)
target = text[:, 1:]

In [19]:
predicted_classes = preds.argmax(dim=-1)

In [20]:
pred_decoded = converter.decode(predicted_classes, length)
pred_decoded[:3]

['BALTHAIAR[s][s][s][s][s][s][s][s][s][s][s]',
 'SIMON[s][s][s][s][s][s][s][s][s][s][s][s][s][s][s]',
 'BENES[s][s][s][s][s][s][s][s][s][s][s][s][s][s][s]']

In [21]:
target_decoded = converter.decode(target, length)
target_decoded[:3]

['BALTHAZAR[s][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO]',
 'SIMON[s][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO]',
 'BENES[s][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO]']

In [22]:
from handwriting_recognition.train import _evaluate

loss_function = CrossEntropyLoss(ignore_index=0).to(get_device())

validation_loss, val_character_error_rate, val_word_error_rate, all_val_preds, all_val_ground_truths = _evaluate(
    epoch=saved_model['epoch'],
    model=model,
    data_loader=val_loader,
    converter=converter,
    loss_function=loss_function
)

Validating Epoch: 24: 100% 624/624 [00:43<00:00, 14.46it/s]


Outputting sample of validation outputs.
['BILEL', 'LAUMONIER', 'JEAN ROCH', 'RUPP', 'PICHON', 'DANIEL', 'JEREMY', 'JEYN MIENEL', 'JULIEN', 'NAEL']
['BILEL', 'LAUMIONIER', 'JEAN ROCH', 'RUPP', 'PICHON', 'DANIEL', 'JEREMY', 'JEAN MICHEL', 'JULIEN', 'NAEL']


In [23]:
val_character_error_rate

tensor(0.0761)

In [24]:
val_word_error_rate

tensor(0.2860)

In [25]:
test_loss, test_character_error_rate, test_word_error_rate, all_test_preds, all_test_ground_truths = _evaluate(
    epoch=saved_model['epoch'],
    model=model,
    data_loader=test_loader,
    converter=converter,
    loss_function=loss_function
)

Validating Epoch: 24: 100% 623/623 [00:42<00:00, 14.57it/s]


Outputting sample of validation outputs.
['KEVIN', 'LOUTTIAL    LI LA', 'LENA', 'JULES', 'CHERPIN', 'MARTIN', 'VALENTINE', 'LORAS', 'THIBAULT', 'AZABI']
['KEVIN', 'CLOTAIRE', 'LENA', 'JULES', 'CHERPIN', 'MARTIN', 'VALENTINE', 'LORAS', 'THIBAULT', 'AZABI']


In [26]:
test_character_error_rate

tensor(0.0756)

In [27]:
test_word_error_rate

tensor(0.2885)